In [1]:
from math import pi, sqrt, exp, cos
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit, fsolve
from scipy import integrate

In [2]:
import matplotlib

matplotlib.rcParams["mathtext.fontset"] = "stix"
matplotlib.rcParams["font.family"] = "STIXGeneral"
matplotlib.rc('font', size=12)

In [3]:
def f_x0(x, n):
    return n

def f_x90(x, j, sigma, sx, sy):
    return j * np.cos(sigma * x + sx) + sy

def find_closest(value, array):
    i = 0
    while array[i] < value:
        i += 1

    return i

In [5]:
x90_d_x0 = []
x90_a_x0 = []

for i in [2, 6, 10, 14, 18, 22, "A2"]:
    if i == "A2":
        x0 = np.loadtxt("lab/cuad/result_A2_X0_22")
        x90 = np.loadtxt("lab/cuad/result_A2_X90_22")
    else:
        x0 = np.loadtxt(f"lab/cuad/result_X0_{i}")
        x90 = np.loadtxt(f"lab/cuad/result_X90_{i}")

    x0 *= 2.0e-2
    x90 *= 2.0e-2

    t = np.arange(0, x0.shape[0] * 2.0e-5, step=2.0e-5)

    (n) = curve_fit(f_x0, t, x0, p0=(1,))[0]
    (j, sigma, sx, sy) = curve_fit(f_x90, t, x90, p0=(0.30, 400, 1, 0.5))[0]

    fig, ax = plt.subplots()
    ax.plot(t, x0, linewidth=0.3)
    ax.plot(t, x90, linewidth=0.3)

    ax.plot(t, np.repeat(n, 2500))
    ax.plot(t, f_x90(t, j, sigma, sx, sy))
    # ax.plot([(2 * pi - sx) / sigma], [f_x90(-sx / sigma, j, sigma, sx, sy)], "o")
    # ax.plot([(4 * pi - sx) / sigma], [f_x90(-sx / sigma, j, sigma, sx, sy)], "o")

    ia = find_closest((2.5 * pi - sx) / sigma, t)
    ib = find_closest((4.5 * pi - sx) / sigma, t)
    ax.plot(t[ia], x90[ia], "o")
    ax.plot(t[ib], x90[ib], "o")

    res_int = integrate.trapezoid(x90[ia:ib], t[ia:ib])
    print(res_int, sy)

    if i != "A2":
        x90_d_x0.append(np.average((x90[ia:ib] / x0[ia:ib]) ** 2))
        x90_a_x0.append(np.average(x90[ia:ib] ** 2 + x0[ia:ib] ** 2))

    plt.figtext(0.15, 0.84, f"$J_0(\\beta)={n[0]:.2f}$")
    plt.figtext(0.15, 0.15, f"$J_1(\\beta)={j / 2:.2f}, \Omega={sigma:.2f}$")

    plt.xlabel("Segundos")
    plt.ylabel("Voltios")

    plt.savefig(f"lab/cuad/plots/plot-{i}.pdf")
    plt.close()

angles = np.array([2, 6, 10, 14, 18, 22])

fig, ax = plt.subplots()
ax.plot(angles, x90_d_x0, "o", label=r"$\langle X_{\pi/2} / {X_0} \rangle$")
plt.legend()

plt.xlabel("Ángulo (sexagesimal) de la lámina retardadora $\lambda/2$")
plt.savefig("lab/cuad/plots/division_average.pdf")
plt.close()

fig, ax = plt.subplots()
ax.plot(angles, x90_a_x0, "o")

x = np.linspace(2, 22)
y = np.sin(4 * (x * np.pi / 180)) ** 2
ax.plot(x, y, label=r"$\sin^2(4\theta)$")

ax.plot(angles, x90_a_x0, "o", label=r"$\langle X_{\pi/2}^2 + X_0^2 \rangle$")
plt.legend()
plt.xlabel("Ángulo (sexagesimal) de la lámina retardadora $\lambda/2$")
plt.savefig("lab/cuad/plots/addition_average.pdf")
plt.close()

0.0106872 0.5496260192014681
0.00047679999999999966 0.03254431192704507
-0.00017240000000000007 -0.012839649989631241
-0.0004601999999999998 -0.04244828335731562
-0.00018139999999999953 -0.017639876751226725
0.0005214000000000004 0.011125680538267358
-2.6000000000006053e-06 -0.04418781034531828
